In [1]:
using ChaosTools
using DynamicalSystems
using FileIO
using JLD2
using LinearAlgebra
using Random
using StaticArrays

In [5]:
a = rand(3,3)

3×3 Matrix{Float64}:
 0.88497    0.673282   0.362362
 0.307727   0.793662   0.342498
 0.0431697  0.0558996  0.413685

In [8]:
a[:,2] /= 2

3-element Vector{Float64}:
 0.08416026638917305
 0.09920773660872192
 0.006987447241892639

In [2]:
include("../src/model/KS.jl")
include("../src/LiftAndLearn.jl")
const LnL = LiftAndLearn

Main.LiftAndLearn

In [ ]:
DATA = load("data/kse_data_L22.jld2");
OPS = load("data/kse_operators_L22.jld2");

In [7]:
@inline @inbounds function kse(x, p, t)
    A = p[1]
    H = p[2]
    N = size(A,1)
    xdot = A*x + H * kron(x,x)
    return SVector{N}(xdot...)
end

@inline @inbounds function kse_jacobian(x, p, t)
    A = p[1]
    H = p[2]
    N = size(A,1)
    J = A + H * kron(1.0I(N), x) + H * kron(x, 1.0I(N))
    return SMatrix{N,N}(J)
end
;

In [10]:
# Settings for the KS equation
KSE = KS(
    [0.0, 22.0], [0.0, 300.0], [1.0, 1.0],
    512, 0.001, 1, "ep"
)

ic_a = [0.8, 1.0, 1.2]
ic_b = [0.2, 0.4, 0.6]

L = KSE.Omega[2] - KSE.Omega[1]  # length of the domain
u0 = (a,b) -> a * cos.((2*π*KSE.x)/L) .+ b * cos.((4*π*KSE.x)/L)  # initial condition
;  

In [16]:
ds = DeterministicIteratedMap(kse, DATA["Vr"][1][:,1:DATA["ro"][end]]' * u0(ic_a[1],ic_b[1]), [OPS["op_int"][1].A, OPS["op_int"][1].H])
tands = TangentDynamicalSystem(ds; J=kse_jacobian)
;

In [19]:
λs = lyapunovspectrum(tands, 1000; Ttr=1000, Δt=0.05, show_progress=true)